<a href="https://colab.research.google.com/github/marcowong02/Show-Recommendation-Project/blob/main/Show%20Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np          # For math functions
import pandas as pd         # For data processing 
import os                   # For working with directories


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

Processing Data 

In [ ]:
# path where the animes dataset is
path = '/content/animes.csv'
df = pd.read_csv(path)

In [ ]:
df

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [ ]:
df.shape

(19311, 12)

Working with the features

In [ ]:
# Choosing the features/columns that I want
selected_features = ['genre']

#X = df[selected_features] # Creates X which is a DataFrame, sorta like a table
#print(X)
#print(df['genre'])

# Since geners look like a list, so we remove the brackets and reformat it
def fix_genre(genre):
    genre = eval(genre)
    genres_without_the_brackets = ' '.join(genre)
    return genres_without_the_brackets 

df['fixed_genre'] = df['genre'].apply(fix_genre)

In [ ]:
df

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,fixed_genre
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,Comedy Sports Drama School Shounen
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,Drama Music Romance School Shounen
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,Sci-Fi Adventure Mystery Drama Fantasy
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,Action Military Adventure Comedy Drama Magic F...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,Action Mystery Supernatural Vampire
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,Sci-Fi Adventure Comedy Magic
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,Adventure Fantasy Magic Martial Arts Comedy Ro...
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,Mystery Horror Supernatural
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,Action Adventure Comedy Historical Demons Supe...


Vectorizer Applied to Fixed Genres

In [ ]:
vectorizer = TfidfVectorizer()
#feature_vectors = vectorizer.fit_transform(combined_features.values.astype('U'))
feature_vectors = vectorizer.fit_transform(df['fixed_genre']) # applies TfidfVectorizer with each respective shows' genres
print(feature_vectors)
#type(feature_vectors)

  (0, 37)	0.43268458369256413
  (0, 33)	0.46689083527108827
  (0, 8)	0.4031020700303863
  (0, 40)	0.5875039399556202
  (0, 5)	0.29520304986313867
  (1, 31)	0.45403199563427893
  (1, 24)	0.4649382596746999
  (1, 37)	0.43647622833768557
  (1, 33)	0.4709822316418675
  (1, 8)	0.4066344810819297
  (2, 10)	0.35898738617418224
  (2, 25)	0.5408725204917503
  (2, 1)	0.3719064484572335
  (2, 11)	0.3856664737551689
  (2, 34)	0.3856664737551689
  (2, 8)	0.3778800190313905
  (3, 20)	0.4329690394185244
  (3, 23)	0.4954624751473473
  (3, 0)	0.28737741811230816
  (3, 10)	0.3096667805067876
  (3, 1)	0.3208109169818726
  (3, 37)	0.3498853446875888
  (3, 8)	0.3259637898656339
  (3, 5)	0.23871250501400004
  (4, 44)	0.6983582657020835
  :	:
  (19307, 10)	0.2384103556763342
  (19307, 1)	0.24699015082382306
  (19307, 31)	0.28020902019228366
  (19307, 8)	0.25095731273561084
  (19307, 5)	0.18378313983707728
  (19308, 16)	0.6555457530409846
  (19308, 42)	0.4839833733719711
  (19308, 25)	0.5796722004455758
  (19

In [ ]:
# getting user-input

# user_input = input('What genres do you like: ')
# print('You put: ' + user_input)

# num_of_shows = input('Number of shows you want to be recommended: ')
# print('You put: ' + num_of_shows)
# num_of_shows = int(num_of_shows)

user_input = 'comedy, sports, drama, school, shounen'
num_of_shows = 10

# find_shows(user_input, num_of_shows)

genre_value = vectorizer.transform([user_input]) # in brackets bc it has to be iterable
#print(genre_value)

Generate CoSine Simlarity Between the Genres and the Feature Genres

In [ ]:
similarity_btwn_animes = cosine_similarity(genre_value, feature_vectors).flatten() # returns how similar the genre is to each and every other genre/feature of shows
similarity_btwn_animes

array([1.        , 0.57266902, 0.15232422, ..., 0.        , 0.29026867,
       0.12976424])

In [ ]:
sorted_index_array = np.argsort(similarity_btwn_animes) # returns array of indces from smallest to largest
sorted_index_array = sorted_index_array[::-1]       # flips the array
sorted_index_array = sorted_index_array[:num_of_shows]  # first n-amounts

shows = df.iloc[sorted_index_array]                   # first n-amounts of shows

shows = shows.sort_values("score", ascending=False)
shows = shows.drop(columns = ['fixed_genre','aired', 'episodes', 'uid', 'members', 'popularity', 'ranked','img_url','link'])

shows.reset_index(drop=True, inplace=True)
shows



,title,synopsis,genre,score
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.82
1,Haikyuu!!,Inspired after watching a volleyball ace nickn...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.62
2,Haikyuu!!,Inspired after watching a volleyball ace nickn...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.62
3,Haikyuu!! Movie 1: Owari to Hajimari,First Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.89
4,Haikyuu!! Movie 1: Owari to Hajimari,First Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.89
5,Haikyuu!! Movie 2: Shousha to Haisha,Second Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.86
6,Rokudenashi Blues,Maeda is a new student in the Teiken High Scho...,"['Sports', 'Comedy', 'School', 'Drama', 'Shoun...",6.11
7,Haikyuu!!: Riku vs. Kuu,"The two-episode anime, adapting chapters from ...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",NaN
8,Haikyuu!!: To the Top,The fourth season of Haikyuu!!,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",NaN
9,Haikyuu!!: To the Top 2nd Season,NaN,"['Comedy', 'Drama', 'School', 'Shounen', 'Spor...",NaN


In [ ]:
def df():
  path = ''
  df = pd.read_csv(path)

def fix_genre(genre):
    genre = eval(genre)
    genres_without_the_brackets = ' '.join(genre)
    return genres_without_the_brackets 

def find_shows(user_input, num_of_shows): 
  
  num_shows = int(num_of_shows)
  
  df['fixed_genre'] = df['genre'].apply(fix_genre)
  vectorizer = TfidfVectorizer()

  feature_vectors = vectorizer.fit_transform(df['fixed_genre'])
  genre_value = vectorizer.transform([user_input])
  similarity_btwn_animes = cosine_similarity(genre_value, feature_vectors).flatten() # returns how similar the genre is to each and every other genre/feature of shows


  sorted_index_array = np.argsort(similarity_btwn_animes) # returns array of indces from smallest to largest
  sorted_index_array = sorted_index_array[::-1]       # flips the array


  sorted_index_array = sorted_index_array[:num_of_shows]  # first n-amounts

  shows = df.iloc[sorted_index_array]                   # first n-amounts of shows
  
  shows = shows.sort_values("score", ascending=False)
  shows = shows.drop(columns = ['fixed_genre','aired', 'episodes', 'uid', 'members', 'popularity', 'ranked','link', 'img_url'])

  shows.reset_index(drop=True, inplace=True)

  return shows

In [ ]:
# getting user-input

# user_input = input('What genres do you like: ')
# print('You put: ' + user_input)

# num_of_shows = input('Number of shows you want to be recommended: ')
# print('You put: ' + num_of_shows)
# num_of_shows = int(num_of_shows)

user_input = 'comedy, sports, drama, school, shounen'
num_of_shows = 10

find_shows(user_input, num_of_shows)


,title,synopsis,genre,score
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.82
735,Haikyuu!!,Inspired after watching a volleyball ace nickn...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.62
17756,Haikyuu!!,Inspired after watching a volleyball ace nickn...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",8.62
86,Haikyuu!! Movie 1: Owari to Hajimari,First Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.89
4342,Haikyuu!! Movie 1: Owari to Hajimari,First Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.89
30,Haikyuu!! Movie 2: Shousha to Haisha,Second Haikyuu!! recap movie.,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",7.86
1391,Rokudenashi Blues,Maeda is a new student in the Teiken High Scho...,"['Sports', 'Comedy', 'School', 'Drama', 'Shoun...",6.11
14986,Haikyuu!!: Riku vs. Kuu,"The two-episode anime, adapting chapters from ...","['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",NaN
14796,Haikyuu!!: To the Top,The fourth season of Haikyuu!!,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",NaN
14794,Haikyuu!!: To the Top 2nd Season,NaN,"['Comedy', 'Drama', 'School', 'Shounen', 'Spor...",NaN


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Image, HTML

user_input = widgets.Text(
    value='',
    placeholder='Type something...',
    description='Anime Genres',
    disabled=False
)

num_of_shows = widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    description='Number of Shows:',
    disabled=False
)

recommendation_list = widgets.Output()

def on_type(data):
  with recommendation_list:
    
      recommendation_list.clear_output()
      if(len(user_input.value) > 1 and (num_of_shows.value > 0)):
        display(find_shows(user_input.value, num_of_shows.value))

num_of_shows.observe(on_type, names='value') 
user_input.observe(on_type, names='value')

display(user_input, num_of_shows, recommendation_list)

Text(value='', description='Anime Genres', placeholder='Type something...')

IntSlider(value=0, description='Number of Shows:', max=10)

Output()